<h2> ADP </h2>

In [122]:
import numpy as np

class Grid_Env: 
	def __init__(self,width,height,start):
		self.width = width
		self.height = height
		self.i = start[0]
		self.j = start[1]

	def set(self,rewards, actions):
		self.rewards = rewards
		self.actions = actions

	def set_state(self,s):
		self.i = s[0]
		self.j = s[1]

	def current_state(self):
		return (self.i,self.j)

	def is_terminal(self,s):
		return s not in self.actions

	def move(self,action):
		'''
		checks if a action is possible, then moves in that direction
		'''
		if action in self.actions[self.i,self.j]:
			if action == 'U':
				self.i -= 1
			elif action == 'D':
				self.i += 1
			elif action == 'R':
				self.j += 1
			elif action == 'L':
				self.j -= 1
		#return reward if any
		return self.rewards.get((self.i,self.j),0)

	def all_states(self):
		return set(list(self.actions.keys()) + list(self.rewards.keys()))

# defining all possible actions that each state can perform
def grid():
    grd = Grid_Env(4, 6,(2,1))
    rewards = {(1, 5): 1, (3, 5): -1, (3,4): -1}
    actions = {
    (0, 0): ('D','R'),
    (0, 1): ('L','D', 'R'),
    (0, 2): ('L','D', 'R'),
    (0, 3): ('L','D', 'R'),
    (0, 4): ('L','D', 'R'),
    (0,5): ('L','D'),
    (1, 0): ('U', 'D','R'),
    (1, 1): ('U', 'D', 'R','L'),
    (1, 2): ('U', 'D', 'R','L'),
    (1, 3): ('U', 'D', 'R','L'),
    (1, 4): ('U', 'D', 'R','L'),
    (1, 5): ('U', 'D','L'),
    (2, 0): ('U', 'R','D'),
    (2, 1): ('U', 'D', 'R','L'),
    (2, 2): ('U', 'D', 'R','L'),
    (2, 3): ('U', 'D', 'R','L'),
    (2, 4): ('U', 'D', 'R','L'),
    (2, 5): ('U', 'D','L'),
    (3, 0): ('U', 'R'),
    (3, 1): ('L', 'R', 'U'),
    (3, 2): ('L', 'R', 'U'),
    (3, 3): ('L', 'R', 'U'),
    (3, 4): ('L', 'R', 'U'),
    (3, 5): ('L', 'U')
    }
    grd.set(rewards, actions)
    return grd


In [123]:
GAMMA = 0.9
ACTIONS = ('U','D','L','R')

SMALL_ENOUGH = 1e-4


if __name__ == '__main__':

    #we use the negative grid so we can make the agent as efficient as possible
    grid = grid()


    #state -> action
    #well randomly choose an action and update as we learn
    policy = {}
    for s in grid.actions.keys():
        policy[s] = np.random.choice(ACTIONS)

    #initialize V(s)
    V = {}
    states = grid.all_states()
    for s in states:
        if s in grid.actions:
            V[s] = np.random.random()
        else:
            #terminal state
            V[s] = 0

    #repeat until convergence
    #V[s] = max[a]{sum[s',r] {p(s',r|s,a)[r + GAMMA * V[s']] } }
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]

            #V[s] only has value if not a terminal state
            if s in policy:
                new_v = float('-inf')

                for a in ACTIONS:
                    grid.set_state(s)
                    r = grid.move(a)
                    v = r + GAMMA * V[grid.current_state()]
                    if v > new_v:
                        new_v = v
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))

        if biggest_change < SMALL_ENOUGH:
            break
    #find a policy that leads to optimal value function
    for s in policy.keys():
        best_a = None
        best_value = float('-inf')
        for a in ACTIONS:
            grid.set_state(s)
            r = grid.move(a)
            v = r + GAMMA * V[grid.current_state()]
            if v > best_value:
                 best_value = v
                 best_a = a
        policy[s] = best_a

In [124]:
l = list(policy.values())
l[11] = 1
l[22] = -1
l[23] = -1

for i in range(24):
    if(l[i] == 'R'):
        l[i] = '→'
    elif(l[i] == 'L'):
        l[i] = '←'
    elif(l[i] == 'U'):
        l[i] = '↑'
    elif(l[i] == 'D'):
        l[i] = '↓'

x = np.array(l)
x = x.reshape(4,6)

In [125]:
from tabulate import tabulate
print(tabulate(x, headers='POLICY', tablefmt='fancy_grid'))

╒═════╤═════╤═════╤═════╤═════╤═════╕
│ P   │ O   │ L   │ I   │ C   │ Y   │
╞═════╪═════╪═════╪═════╪═════╪═════╡
│ ↓   │ ↓   │ ↓   │ ↓   │ ↓   │ ↓   │
├─────┼─────┼─────┼─────┼─────┼─────┤
│ →   │ →   │ →   │ →   │ →   │ 1   │
├─────┼─────┼─────┼─────┼─────┼─────┤
│ ↑   │ ↑   │ ↑   │ ↑   │ ↑   │ ↑   │
├─────┼─────┼─────┼─────┼─────┼─────┤
│ ↑   │ ↑   │ ↑   │ ↑   │ -1  │ -1  │
╘═════╧═════╧═════╧═════╧═════╧═════╛


In [126]:
#Get shortest path
def get_shortest_path(i,j):
    start_point = (i,j)
    path = [start_point]
    while True:
        if x[i][j] == '1':
            break
        if x[i][j] == "↓":
            i = i+1
            path.append((i,j))
        elif x[i][j] == "↑":
            i = i-1
            path.append((i,j))
        elif x[i][j] == "→":
            j = j+1
            path.append((i,j))
        else:
            j = j-1
            path.append((i,j))
    return path

In [127]:
print(get_shortest_path(0,0))

[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]


In [128]:
print(get_shortest_path(2,1))

[(2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
